# Logistic Regression - Stochastic Gradient Descent

The goal of this notebook is to study the following two aspects of the Gradient Descent algorithm for Multi-class Logistic Regression.

- Stochastic Gradient Descent 
- Regularization via Early Stopping


### <font color=red> Early Stopping</font>

Early stopping is a regularization technique for iterative optimization algorithms such as Graient Descent that stops training as soon as the validation error reaches a minimum. 

Because we start with weights almost zero and they move away as training continues, stopping early corresponds to a model with more weights close to zero and effectively fewer parameters.

### Early Stopping Curve:

With Stochastic (and Mini-batch) Gradient Descent, the error vs iterations (epochs) curves are not so smooth.

It may be hard to know whether we have reached the minimum or not. 

One solution is to stop only after the validation error has been above the minimum for some time (when we are confident that the model will not do any better), then roll back the model parameters to the point where the validation error was at a minimum.

More on Stochastic Gradient Descent:
https://scikit-learn.org/stable/modules/sgd.html#sgd

# Dataset


We will use the iris dataset, which is a multivariate data set. 

This is a famous dataset that contains the sepal and petal length and width of 150 iris flowers of three different species: Iris-Setosa, Iris-Versicolor, and Iris-Virginica

There are 4 features: 
- sepal length (cm)
- sepal width (cm)
- petal length (cm)
- petal width (cm)

Total number of samples: 150

The dataset is also known as Fisher's Iris data set as it was introduced by the British statistician and biologist Ronald Fisher in his 1936 paper "The use of multiple measurements in taxonomic problems as an example of linear discriminant analysis".


<img src="https://cse.unl.edu/~hasan/IrisFlowers.png" width=800, height=400>

In [1]:
import warnings
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

## Explore The Dataset

In [2]:
iris = load_iris()

print(iris.keys())
print(iris.feature_names)
print(iris.target_names)
print(iris.data.shape)

#print(iris.DESCR)

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']
(150, 4)


## Create Data Matrix (X) and the Label Vector (y)

We can use all features or a subset. For this notebook, we will use two features (i.e., petal length, petal width).

In [3]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

## Split Data Into Training and Test Sets

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Stochastic Gradient Descent


The main problem with Batch Gradient Descent is the fact that it uses the whole training set to compute the gradients at every step, which makes it very slow when the training set is large. 

At the opposite extreme, Stochastic Gradient Descent just picks a random instance in the training set at every step and computes the gradients based only on that single instance. 

Obviously this makes the algorithm much faster since it has very little data to manipulate at every iteration. It also makes it possible to train on huge training sets, since only one instance needs to be in memory at each iteration.

On the other hand, due to its stochastic (i.e., random) nature, this algorithm is much less regular than Batch Gradient Descent: instead of gently decreasing until it reaches the minimum, the cost function will bounce up and down, decreasing only on average. 

Over time it will end up very close to the minimum, but once it gets there it will continue to bounce around, never settling down. So once the algorithm stops, the final parameter values are good, but not optimal.


## Scikit-Learn SGDClassifier


The SGDClassifier implements a plain stochastic gradient descent learning routine which supports different loss functions and penalties for classification.

The concrete loss function can be set via the loss parameter. SGDClassifier supports the following loss functions:

- loss="hinge": (soft-margin) linear Support Vector Machine
- loss="modified_huber": smoothed hinge loss
- loss="log": logistic regression

For implementing SGD for Logistic Regression, we usually use the "log" loss. The "log" loss gives logistic regression, a probabilistic classifier.

Using loss="log" enables the predict_proba method, which gives a vector of probability estimates per sample.



We need to set the following attributes to train a SGDClassifier.


- penalty : ‘none’, ‘l2’, ‘l1’, or ‘elasticnet’
    -- The penalty (aka regularization term) to be used. Defaults to ‘l2’ which is the standard regularizer for linear SVM models. ‘l1’ and ‘elasticnet’ might bring sparsity to the model (feature selection) not achievable with ‘l2’.
    

- alpha : Constant that multiplies the regularization term. Defaults to 0.0001 


- l1_ratio : The Elastic Net mixing parameter, with 0 <= l1_ratio <= 1. l1_ratio=0 corresponds to L2 penalty, l1_ratio=1 to L1. Defaults to 0.15.


- max_iter : The maximum number of passes over the training data (aka epochs). It only impacts the behavior in the fit method, and not the partial_fit. Defaults to 5. Defaults to 1000 from 0.21, or if tol is not None.


- tol : The stopping criterion. If it is not None, the iterations will stop when (loss > previous_loss - tol). Defaults to 1e-3 from 0.21.


- random_state : The seed of the pseudo random number generator to use when shuffling the data. If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.


- learning_rate : The learning rate schedule:

    -- ‘constant’: eta = eta0

    --‘optimal’: [default] eta = 1.0 / (alpha * (t + t0)) where t0 is chosen by a heuristic proposed by Leon Bottou.

    --‘invscaling’: eta = eta0 / pow(t, power_t)

    --‘adaptive’: eta = eta0, as long as the training keeps decreasing. Each time n_iter_no_change consecutive epochs fail to decrease the training loss by tol or fail to increase validation score by tol if early_stopping is True, the current learning rate is divided by 5.


- eta0 : The initial learning rate for the ‘constant’, ‘invscaling’ or ‘adaptive’ schedules. The default value is 0.0 as eta0 is not used by the default schedule ‘optimal’.



- early_stopping : Whether to use early stopping to terminate training when validation score is not improving. If set to True, it will automatically set aside a fraction of training data as validation and terminate training when validation score is not improving by at least tol for n_iter_no_change consecutive epochs.


- n_iter_no_change : Number of iterations with no improvement to wait before early stopping.

More detail: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


## SGDClassifier for Multi-Class Classification

SGDClassifier supports multi-class classification by combining multiple binary classifiers in a “one versus all” (OvA) scheme. 

For each of the  classes, a binary classifier is learned that discriminates between that and all other classes. 

At testing time, we compute the confidence score (i.e. the signed distances to the hyperplane) for each classifier and choose the class with the highest confidence. 

Note that the Logistic Regression SGDClassifier **does not use the Softmax regression** technique for multi-class classification.

## Investigation of the Stochastic Gradient Descent for Logistic Regression

We will investigate the SGDClassifier:
- Without Early Stopping
- With Early Stopping

## Stochastic Gradient Descent Without Early Stopping


We will implement the SGDClassifier without early stopping.

For larger dataset, even with l2 regularization in effect, the weights will be larger.


First, we need to find the optimal hyperparameters via Gridsearch.

## Model Selection: Hyperparameter Tuning

In [5]:
%%time

warnings.filterwarnings('ignore')


param_grid = {'alpha': [0.05, 0.01, 0.001],
              'penalty' : ["l2"],
              'learning_rate': ["constant", "optimal", "invscaling", "adaptive"], 
              'max_iter':[500, 1000, 3000, 7000],
              'eta0': [0.1, 0.01, 0.001],
              'tol': [1e-3, 1e-5, 1e-8],
              'loss': ['hinge', 'log', 'modified_huber']}

  
sgd_clf = SGDClassifier()

sgd_clf_cv = GridSearchCV(sgd_clf, param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
sgd_clf_cv.fit(X_train, y_train)

params_optimal = sgd_clf_cv.best_params_

print("Best Score (accuracy): %f" % sgd_clf_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal)
print("\n")

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s


Best Score (accuracy): 0.966667
Optimal Hyperparameter Values:  {'alpha': 0.05, 'eta0': 0.1, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'max_iter': 3000, 'penalty': 'l2', 'tol': 1e-05}


CPU times: user 3.56 s, sys: 145 ms, total: 3.71 s
Wall time: 13.2 s


[Parallel(n_jobs=-1)]: Done 3888 out of 3888 | elapsed:   13.2s finished


## Train the Optimal SGDClassifier without Early Stopping

In [6]:
sgd = SGDClassifier(**params_optimal)
sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.1, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=3000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=1e-05,
              validation_fraction=0.1, verbose=0, warm_start=False)

## Evaluate the Optimal SGDClassifier without Early Stopping on Test Data

In [7]:
print("\nNo. of Iterations:", sgd.n_iter_ )

print("\nWeight Coefficients:\n", sgd.coef_ )

print("\nWeight Intercept:\n", sgd.intercept_ )


y_test_predict = sgd.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predict == y_test)
print("\nTest Accuracy: ", accuracy_score_test)

# Confusion Matrix
print("\nTest Confusion Matrix (Test Data):\n", confusion_matrix(y_test, y_test_predict))


No. of Iterations: 29

Weight Coefficients:
 [[-3.56035391 -1.28575359]
 [ 0.56404723 -0.85165433]
 [ 2.49510599  1.63625871]]

Weight Intercept:
 [ 13.09045347  -1.040835   -14.33817042]

Test Accuracy:  1.0

Test Confusion Matrix (Test Data):
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


## Stochastic Gradient Descent With Early Stopping


We will implement the Logistic Regression SGDClassifier with early stopping.

First, we need to find the optimal hyperparameters via Gridsearch.


## Model Selection: Hyperparameter Tuning

In [8]:
%%time

warnings.filterwarnings('ignore')

param_grid = {'alpha': [0.05, 0.01, 0.001],
              'penalty' : ["l2", "l1"],
              'learning_rate': ["constant", "optimal", "invscaling", "adaptive"], 
              'max_iter':[100, 500, 1000, 3000, 7000],
              'eta0': [0.01, 0.001],
              'tol': [1e-3, 1e-5, 1e-8],
              'loss': ['hinge', 'log', 'modified_huber']}


sgd_clf = SGDClassifier(early_stopping=True)

sgd_clf_cv = GridSearchCV(sgd_clf, param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
sgd_clf_cv.fit(X_train, y_train)

params_optimal = sgd_clf_cv.best_params_

print("Best Score (accuracy): %f" % sgd_clf_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal)
print("\n")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 2160 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Done 340 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 2185 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 4435 tasks      | elapsed:   16.8s


Best Score (accuracy): 0.950000
Optimal Hyperparameter Values:  {'alpha': 0.01, 'eta0': 0.01, 'learning_rate': 'optimal', 'loss': 'log', 'max_iter': 500, 'penalty': 'l2', 'tol': 1e-08}


CPU times: user 7.29 s, sys: 176 ms, total: 7.47 s
Wall time: 24.2 s


[Parallel(n_jobs=-1)]: Done 6480 out of 6480 | elapsed:   24.1s finished


## Train the Optimal SGDClassifier with Early Stopping

In [9]:
sgd_early_stopping = SGDClassifier(**params_optimal, early_stopping=True)


sgd_early_stopping.fit(X_train, y_train)

SGDClassifier(alpha=0.01, average=False, class_weight=None, early_stopping=True,
              epsilon=0.1, eta0=0.01, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='log', max_iter=500,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=1e-08,
              validation_fraction=0.1, verbose=0, warm_start=False)

## Evaluate the Optimal SGDClassifier with Early Stopping on Test Data

In [10]:
print("\nNo. of Iterations:", sgd_early_stopping.n_iter_ )


print("\nWeight Coefficients:\n", sgd_early_stopping.coef_ )

print("\nWeight Intercept:\n", sgd_early_stopping.intercept_ )


y_test_predict = sgd_early_stopping.predict(X_test)
#print(y_test_predict)

accuracy_score_test = np.mean(y_test_predict == y_test)
print("\nTest Accuracy: ", accuracy_score_test)

# Confusion Matrix
print("\nTest Confusion Matrix (Test Data):\n", confusion_matrix(y_test, y_test_predict))


No. of Iterations: 11

Weight Coefficients:
 [[-2.37060705 -1.10610181]
 [ 0.97566343 -1.50170909]
 [ 2.41244288  2.80636982]]

Weight Intercept:
 [  7.29595672  -2.30367483 -16.68979844]

Test Accuracy:  1.0

Test Confusion Matrix (Test Data):
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


## Comparision: Early Stopping & Without Early Stopping

In [11]:

print("\nWeight Coefficients (Without Early Stopping):\n", sgd.coef_ )
print("\nWeight Coefficients (Early Stopping):\n", sgd_early_stopping.coef_)

print("\n")
print("\nNo. of Iterations (Without Early Stopping):", sgd.n_iter_ )
print("\nNo. of Iterations (Early Stopping):", sgd_early_stopping.n_iter_ )


Weight Coefficients (Without Early Stopping):
 [[-3.56035391 -1.28575359]
 [ 0.56404723 -0.85165433]
 [ 2.49510599  1.63625871]]

Weight Coefficients (Early Stopping):
 [[-2.37060705 -1.10610181]
 [ 0.97566343 -1.50170909]
 [ 2.41244288  2.80636982]]



No. of Iterations (Without Early Stopping): 29

No. of Iterations (Early Stopping): 11


## Observation: Early Stopping Regularization

The SGDClassifier (or any gradient descent algorithm) starts with weights almost zero. Then, it moves away as training continues. If we stop the training early, as soon as the validation error reaches minimum, we get a model with more weights close to zero and effectively fewer parameters.

If we compare the weight coefficients and the number iterations between the two implementations (without and with early stopping), we will see the difference. 

Early stopping makes the weights smaller (enables feature selection) and requires less iterations (faster training time). 

However, due to its stochastic nature we will not always get the optimal theta. Hence the model performance will not be as good as the gradient descent based approach.

With smaller dataset (e.g., Iris), we will not see the benefit of SGDclassifier with early stopping. However, for larger datasets (e.g., MNIST handwritten digit recongnition), the SGDClassifier will improve the model performance in two ways:

- Faster training time
- Feature selection (due to early stopping as well as l2 regularization)
